In [274]:
import pandas as pd
import re
import datetime

# Define board name
board_name = 'mobilesales|MacShop'

# Define encoding
ecode = 'utf-8-sig'

# Define keyword
keyword = 'ipadmini系列報價'

# Define regular expression patterns
#mobilesales filter
pattern1 = r'欲售價格：[^\d]*([\d,]+|售價：([\d,]{4,})|\[售價\]\s*NTD\s*(\d+)\s*|\[售價\]\d+|\[售價\]\s*(\d+)|\[售價\]\s*NT\$([\d,]+)|\[欲售價格\]：\s*(\d+))|\欲售價格\s*:(\d{3,})|\[售價\]\s*(\d+)|售價：([\d,]{4,})| \【售價\】\s*(\d+)|\[售價\]\s*NT\$\s*(\d+)|\[售價\]\s*(?:NT\$)?([$]?[\d,]+)'
#MacShop filter
pattern2 = r'\[售價\]\s*(?:NT\$)?([$]?[\d,]+)'
#search keywords
pattern3 = r'ipad\s*(?:mini\s*[23456])'
#price filter
pattern4 = r'\n[123456789].s*\d{3,}'

# Read the CSV file
df = pd.read_csv(r'C:\Users\a3343\Desktop\ptt_data\ptt_手機平板完整資料_0303.csv')
df = df[['作者', '版名', '標題', '內文', '日期']]

# Filter rows based on conditions
df = df[df['標題'].str.contains('賣|售|\[販售\]') & df['版名'].str.contains(board_name)]

# Remove commas from '內文'
df['內文'] = df['內文'].str.replace(',', '')
df['內文'] = df['內文'].astype(str)

    
# extract price
df['售價'] = pd.to_numeric(df['內文'].str.extract(pattern1).fillna('').sum(axis=1), errors='coerce')

# # Reorder columns and show the df
df = df[['作者', '版名', '標題', '售價', '內文', '日期']]

# Filter out rows where '售價' is not numeric
df = df[~pd.to_numeric(df['售價'], errors='coerce').isnull()]

# Filter by specific model of product
df = df.loc[df['標題'].str.contains(pattern3, na=False, case=False, regex=True)].sort_values(by='售價', ascending=True)

# Save to CSV
df.to_csv(f'C:/Users/a3343/Downloads/ptt_{keyword}_報價test4_{datetime.date.today()}.csv', encoding=ecode, index=False)
